In [5]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
import math
#import random
import numpy as np
from scipy.optimize import minimize
from qiskit.providers.aer import QasmSimulator
from qiskit.aqua.components.optimizers import COBYLA,SPSA
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
#from qiskit.providers.aer.noise import pauli_error
#from qiskit.providers.aer.noise import depolarizing_error
from qiskit.test.mock import FakeVigo
from qiskit.tools.monitor import job_monitor



In [6]:
def create_circuit(parameters):
    
    qctl = QuantumRegister(2)
    qc = ClassicalRegister(2)
    circ = QuantumCircuit(qctl, qc)

    circ.rx(parameters[0],qctl[0])
    circ.ry(parameters[1],qctl[0])
    circ.rx(parameters[2],qctl[1])
    circ.cx(qctl[0],qctl[1])
    
    circ.measure(qctl,qc)
    
    return circ

In [26]:
def calculate_cost_function(parameters):
    
    device_backend = FakeVigo()
    coupling_map = device_backend.configuration().coupling_map
    
    phase=0
    probability={}
    times=1000
    
    simulator = QasmSimulator()
    
    #p_gate1 = 0.005
    #p_meas = 0.005
    
    circ = create_circuit(parameters)
    
    #error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    #error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    #error_gate2 = error_gate1.tensor(error_gate1)
    
    #noise_model= NoiseModel()
    
    #noise_model.add_all_qubit_quantum_error(error_meas, "measure")
    #noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"])
    #noise_model.add_all_qubit_quantum_error(error_gate1, ["ry", "rx"])
    
    noise_model = NoiseModel.from_backend(device_backend)
    job = execute(circ, simulator,
                basis_gates = noise_model.basis_gates,
                noise_model = noise_model,
                shots = times)    
    counts = job.result().get_counts()
    
    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0
        
    for i in counts.keys():
        probability[i] = counts[i]/times
    
    rev_phase= (np.cos(parameters[0]/2)*np.sin(parameters[1]/2) - 1j*np.sin(parameters[0]/2)*np.cos(parameters[1]/2)) / (np.cos(parameters[0]/2)*np.cos(parameters[1]/2) + 1j*np.sin(parameters[0]/2)*np.sin(parameters[1]/2))
    
    print (probability,rev_phase)
    return ((probability['00'])**2 + (probability['01']-1/2)**2 + (probability['10']-1/2)**2 + (probability['11'])**2)+ (abs(rev_phase-np.exp(1j*phase)))**2  

In [27]:
#out = minimize(calculate_cost_function, x0=[0,0,0], method="COBYLA", options={'maxiter':200})
optimizer = SPSA(maxiter = 300)
initial = [0,0,0]
bounds = [(0,2*np.pi), (0,2*np.pi), (0,2*np.pi)]
outcome = optimizer.optimize(num_vars=3, objective_function=calculate_cost_function, initial_point=initial, variable_bounds=bounds)
print(outcome)


{'00': 0.974, '01': 0.008, '10': 0.014, '11': 0.004} (0.04991608228393454+0.0501667068599526j)
{'00': 0.974, '01': 0.01, '10': 0.011, '11': 0.005} (-0.04991608228393454-0.0501667068599526j)
{'00': 0.981, '01': 0.008, '10': 0.009, '11': 0.002} (0.04991608228393454-0.0501667068599526j)
{'00': 0.979, '01': 0.007, '10': 0.009, '11': 0.005} (-0.04991608228393454+0.0501667068599526j)
{'00': 0.977, '01': 0.007, '10': 0.008, '11': 0.008} (-0.04991608228393454+0.0501667068599526j)
{'00': 0.981, '01': 0.008, '10': 0.005, '11': 0.006} (0.04991608228393454-0.0501667068599526j)
{'00': 0.972, '01': 0.01, '10': 0.016, '11': 0.002} (-0.04991608228393454+0.0501667068599526j)
{'00': 0.981, '01': 0.005, '10': 0.005, '11': 0.009} (0.04991608228393454-0.0501667068599526j)
{'00': 0.977, '01': 0.006, '10': 0.009, '11': 0.008} (0.04991608228393454-0.0501667068599526j)
{'00': 0.976, '01': 0.005, '10': 0.013, '11': 0.006} (-0.04991608228393454+0.0501667068599526j)
{'00': 0.973, '01': 0.006, '10': 0.018, '11': 0

In [10]:
help(SPSA)


Help on class SPSA in module qiskit.aqua.components.optimizers.spsa:

class SPSA(qiskit.aqua.components.optimizers.optimizer.Optimizer)
 |  SPSA(maxiter: int = 1000, save_steps: int = 1, last_avg: int = 1, c0: float = 0.6283185307179586, c1: float = 0.1, c2: float = 0.602, c3: float = 0.101, c4: float = 0, skip_calibration: float = False, max_trials: Union[int, NoneType] = None) -> None
 |  
 |  Simultaneous Perturbation Stochastic Approximation (SPSA) optimizer.
 |  
 |  SPSA is an algorithmic method for optimizing systems with multiple unknown parameters.
 |  As an optimization method, it is appropriately suited to large-scale population models,
 |  adaptive modeling, and simulation optimization.
 |  
 |  .. seealso::
 |      Many examples are presented at the `SPSA Web site <http://www.jhuapl.edu/SPSA>`__.
 |  
 |  SPSA is a descent method capable of finding global minima,
 |  sharing this property with other methods as simulated annealing.
 |  Its main feature is the gradient appro